In [18]:
%matplotlib inline

In [19]:
import os
imbd_dir='aclImdb/aclImdb'
train_dir=os.path.join(imbd_dir,'train')
labels=[]
texts=[]

In [20]:
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname),errors='ignore',encoding='gb18030')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen=100
training_samples=200
validation_samples=10000
max_words=10000

In [22]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)

In [23]:
word_index=tokenizer.word_index

In [24]:
data=pad_sequences(sequences,maxlen=maxlen)
labels=np.array(labels)
print(data.shape)
print(labels.shape)

(25000, 100)
(25000,)


In [25]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [26]:
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:training_samples + validation_samples]
y_val = labels[training_samples:training_samples + validation_samples]

In [27]:
glove_dir='glove.6B'
embedding_index={}
f=open(os.path.join(glove_dir,'glove.6B.100d.txt'),encoding='gb18030',errors='ignore')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embedding_index[word]=coefs
f.close()
print(len(embedding_index))

399917


In [29]:
embedding_dim=100
embedding_matrix=np.zeros((max_words,embedding_dim))
for word,i in word_index.items():
    if i<max_words:
        embedding_vector=embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector

In [30]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [32]:
from keras import losses,activations,optimizers
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable=False

In [33]:
model.compile(
    optimizer=optimizers.RMSprop(0.0001),
    loss=losses.binary_crossentropy,
    metrics=['acc']
)

In [34]:
model.fit(x_train,y_train,
          epochs=10,
          batch_size=32,
          validation_data=(x_val,y_val)
          )

Instructions for updating:
Use tf.cast instead.


Train on 200 samples, validate on 10000 samples
Epoch 1/10


 32/200 [===>..........................] - ETA: 2s - loss: 0.6682 - acc: 0.6250

200/200 [==============================] - 1s 5ms/step - loss: 0.7132 - acc: 0.5550 - val_loss: 0.7143 - val_acc: 0.5124


Epoch 2/10
 32/200 [===>..........................] - ETA: 0s - loss: 0.5447 - acc: 0.8125

...

200/200 [==============================] - 0s 2ms/step - loss: 0.1559 - acc: 1.0000 - val_loss: 0.7528 - val_acc: 0.5455


Epoch 9/10
 32/200 [===>..........................] - ETA: 0s - loss: 0.1118 - acc: 1.0000

200/200 [==============================] - 0s 2ms/step - loss: 0.1221 - acc: 1.0000 - val_loss: 0.7838 - val_acc: 0.5431


Epoch 10/10
 32/200 [===>..........................] - ETA: 0s - loss: 0.1063 - acc: 1.0000

200/200 [==============================] - 0s 2ms/step - loss: 0.1004 - acc: 1.0000 - val_loss: 0.7230 - val_acc: 0.5566
